In [105]:
from tkinter import W
from trace import Trace
import pandas as pd
import numpy as np
import nltk
import pickle
import csv
#nltk.download()

from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, accuracy_score, f1_score, confusion_matrix, recall_score

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ufukp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ufukp\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\ufukp\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\ufukp\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ufukp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [106]:
import pickle
hate_data, hate_labels = [], [] 
with open("hate_dataset", "rb") as fp:     
    hate_data= pickle.load(fp)
with open("hate_labels", "rb") as fp:     
    hate_labels= pickle.load(fp)

#print(hate_data)


In [107]:
hate_data = pd.DataFrame(hate_data)

hate_data[0].dropna(inplace=True)
hate_data[0] = [entry.lower() for entry in hate_data[0]] # all lowercase
hate_data[0]= [word_tokenize(entry) for entry in hate_data[0]] # tokenization
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ #seperate adjectives
tag_map['V'] = wn.VERB #seperate verbs
tag_map['R'] = wn.ADV #seperate adverbs

In [108]:
for index,entry in enumerate(hate_data[0]):
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    hate_data.loc[index,'text_final'] = str(Final_words)
    
print("Finished preprocessing")
    

Finished preprocessing


In [76]:
print(hate_data['text_final'])

0        ['hurray', 'save', 'u', 'many', 'way', 'potus'...
1        ['would', 'young', 'fight', 'age', 'men', 'vas...
2        ['kamalaharris', 'illegals', 'dump', 'kid', 'b...
3        ['ny', 'time', 'white', 'state', 'pose', 'arra...
4        ['orban', 'brussels', 'european', 'leader', 'i...
                               ...                        
26810                   ['cry', 'hard', 'waifu', 'faggot']
26811    ['oh', 'boy', 'come', 'long', 'venting', 'post...
26812    ['dont', 'know', 'exactly', 'feel', 'course', ...
26813    ['op', 'stop', 'faggot', 'post', 'video', 'nex...
26814    ['clue', 'whos', 'point', 'time', 'need', 'get...
Name: text_final, Length: 26815, dtype: object


In [109]:
hate_labels = pd.DataFrame(hate_labels)

In [110]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(hate_data['text_final'],hate_labels[0],test_size=0.3)
print(Test_X)
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)
print(Train_X.shape)
print(Train_Y.shape)

20283    ['recently', 'get', 'recommendation', 'peacefu...
12850                                             ['true']
25104                        ['hoist', 'retard', 'edness']
21864    ['nothing', 'infuriates', 'retard', 'try', 'in...
9808                        ['ai', 'bitch', 'try', 'rape']
                               ...                        
16088    ['feature', 'pretty', 'good', 'cut', 'edge', '...
1552     ['arrested', 'gangster', 'alien', 'child', 'br...
16163    ['wan', 'na', 'shake', 'anytime', 'new', 'mili...
23500    ['would', 'even', 'matter', 'law', 'afford', '...
11493                                ['justify', 'policy']
Name: text_final, Length: 8045, dtype: object
(18770,)
(18770,)


In [111]:
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(hate_data['text_final'])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)
print(Test_X)

20283    ['recently', 'get', 'recommendation', 'peacefu...
12850                                             ['true']
25104                        ['hoist', 'retard', 'edness']
21864    ['nothing', 'infuriates', 'retard', 'try', 'in...
9808                        ['ai', 'bitch', 'try', 'rape']
                               ...                        
16088    ['feature', 'pretty', 'good', 'cut', 'edge', '...
1552     ['arrested', 'gangster', 'alien', 'child', 'br...
16163    ['wan', 'na', 'shake', 'anytime', 'new', 'mili...
23500    ['would', 'even', 'matter', 'law', 'afford', '...
11493                                ['justify', 'policy']
Name: text_final, Length: 8045, dtype: object


In [112]:
Naive = naive_bayes.MultinomialNB()
Naive.fit(Train_X_Tfidf,Train_Y)
predictions_NB = Naive.predict(Test_X_Tfidf)
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, Test_Y)*100)

Naive Bayes Accuracy Score ->  82.16283405842137


In [ ]:
#SVM
#print(Train_Y.shape)
# gamma = 2**-4, 2**-3, 2**-2, 2**-1, 2**0, 2**1
# c = 10**-3, 10**-2, 10**-1, 10**0, 10**1
SVM = svm.SVC(C=1.0, kernel='rbf', degree=3, gamma=2**1)
SVM.fit(Train_X_Tfidf,Train_Y)
predictions_SVM = SVM.predict(Test_X_Tfidf)
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)
print("Precision -> ", precision_score(Test_Y, predictions_SVM, average="binary"))
print("Recall -> ", recall_score(Test_Y, predictions_SVM, average="binary"))
print("F1 Score -> ", f1_score(Test_Y, predictions_SVM, average="binary"))
cm = confusion_matrix(Test_Y, predictions_SVM, labels=[0, 1])
print("Confusion Matrix -> ", cm)

tn, fp, fn, tp = confusion_matrix(list(Test_Y), list(predictions_SVM), labels=[0, 1]).ravel()
print('True Positive', tp)
print('True Negative', tn)
print('False Positive', fp)
print('False Negative', fn)

npv = tn / (tn+fn)
print("NPV -> ", npv)
fpr = fp / (fp+tn)
print("FPR -> ", fpr)
fdr = fp / (tp+fp)
print("FDR -> ", fdr)

# filename = 'finalized_model.sav'
# pickle.dump(predictions_SVM, open(filename, 'wb'))

In [103]:
sentence = "trans women are real women" #input("Please enter a sentence:")
hate_data2 = []
hate_data2.append(sentence)

hate_data2 = pd.DataFrame(hate_data2)

hate_data2[0].dropna(inplace=True)
hate_data2[0] = [entry.lower() for entry in hate_data2[0]] # all lowercase
hate_data2[0]= [word_tokenize(entry) for entry in hate_data2[0]] # tokenization
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ #seperate adjectives
tag_map['V'] = wn.VERB #seperate verbs
tag_map['R'] = wn.ADV #seperate adverbs

for index,entry in enumerate(hate_data2[0]):
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    hate_data2.loc[index,'text_final'] = str(Final_words)
    
print("Finished preprocessing")

print(hate_data2)

Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(hate_data['text_final'])
Test_X_Tfidf = Tfidf_vect.transform(hate_data2['text_final'])
print(Test_X_Tfidf.shape)
predictions_SVM = SVM.predict(Test_X_Tfidf)
print(predictions_SVM)

Finished preprocessing
                                  0                           text_final
0  [trans, women, are, real, women]  ['trans', 'woman', 'real', 'woman']
(1, 5000)
[1]
